In [1]:
import pandas as pd

In [2]:
def pickled_df_loader (year, columns):
    """
    loads dataframes from the pickled dfs. 
    """

    df= pd.read_pickle(f"./data_deutsches_zeitungsportal_1914_1945/newspapers_ger_{year}")[columns]
    return df

In [3]:
# def challenge_df_generator (begin, end):
#     """
#     loads all pickled dataframes belonging to the years in the year range and returns their concatenation as a single df. 
#     """ 
    
#     columns= ['paper_title', 'publication_date', 'place_of_distribution']
    
#     df= pickled_df_loader (begin, columns)
    
#     for year in range (begin+1, end+1):
#         df= pd.concat([df, pickled_df_loader(year, columns)], axis=0)
        
#     return df

In [4]:
#newspapers_1914_1945_df = challenge_df_generator (1914, 1946)
#newspapers_1914_1945_df.to_pickle("./data_deutsches_zeitungsportal_1914_1945/newspapers_1914_1945_df")

Doing some entitiy recognition on the first article of the first newspaper in the first df, using spacy: 

In [5]:
df= pd.read_pickle ("./data_deutsches_zeitungsportal_1914_1945/newspapers_ger_1914_part_1")

In [19]:
! python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 310.6 kB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [23]:
import spacy
nlp = spacy.load("de_core_news_sm")
doc = nlp(df.iloc[0,13])

for ent in doc.ents: 
    """
    Getting the names of all locations within the loaded newspaper article:
    """
    if ent.label_== 'LOC':
        print(ent.text)

Haake
H
Widdig
Bensberg
Textbreite
Herrn Kardinal Wege
Paderborn
Briefes
Schle
Anschuldigun
Adres Teil
Hedwigskirche
Gedächtnisrede
Rom
“
Fürstbistum Breslau
Paderborn
Regierungs
goldene Straße
Veröffentlichung
Mittelwegs
Elsaß
Herrn Erzbischof von Köln
Elsaß
Pagen
Schlechter Kamerad
Damen
Vorstellens
Estelle
Voss
Berlin
Bayern
England
Luftflottenfrage
England
England
England
Deutschland
Staaten
Deutschland
England
Frankreich
Sozi
Wintergartens
Chinesen
Weißt du
Sylvies
Lachen


Let's see whether "Widdig" from the above list is really a location:

In [26]:
import wikipedia as wiki
wiki.set_lang("de")

search_string = 'Widdig'
page = wiki.page(search_string)
content = page.content
content[0:680]

'Widdig ist ein Stadtteil von Bornheim im Rhein-Sieg-Kreis von Nordrhein-Westfalen.\n\n\n== Geographie ==\nWiddig liegt im Zentrum der Kölner Bucht, zwischen den beiden Großstädten Köln und Bonn. Es grenzt im Osten unmittelbar ans linke Ufer des Rheins, im Norden an die Stadt Wesseling mit ihrem Stadtteil Urfeld, im Westen an den Bornheimer Stadtteil Sechtem sowie im Süden an den Bornheimer Stadtteil Uedorf. Auf dem gegenüberliegenden rechten Rheinufer liegt die Stadt Niederkassel mit ihrem Stadtteil Rheidt. Westlich des Ortskerns befinden sich die Landesstraße 300 (vormals Bundesstraße 9) und die Rheinuferbahn, noch weiter westlich die Bundesautobahn 555.\n\n\n== Geschichte ==\nD'

Extracting names of cities from the newspaper_df:

In [7]:
# def city_extractor(newspaper_df):
# #extracts names of cities from the dataframe containing the full texts.

#     city_list=[]

#     for place in newspaper_df['place_of_distribution']:
#         if isinstance (place, list):
#             if place[0] not in city_list:
#                 city_list.append(place[0])
#         else: 
#             if place not in city_list:
#                 city_list.append(place)

#     return city_list

In [8]:
# cities= city_extractor(df)
# cities

Pinning the cities from a city list on the world map:

In [9]:
# def map_maker(city_list):
# #plots the cities in the city list on a map.
    
#     city_dict= {}
#     gn= geocoders.GeoNames(username="golisf")
    
#     for city in city_list:
#         if not pd.isna(city):
#             city_dict[city]= gn.geocode(city)
            
#     # Create a map centered at a location, you can adjust the coordinates and zoom level as needed
#     map_center= [51.1657, 10.4515]  # Germany's approximate center
#     my_map= folium.Map(location=map_center, zoom_start=6)
 
#     # Add markers for each city
#     for city in list(city_dict.keys()):
#         folium.Marker(location=city_dict[city], popup=city).add_to(my_map)
        
#     return my_map

In [10]:
#map_maker(cities)